In [17]:
import _pickle as cPickle
import gzip

import numpy as np


def load_data():
    f = gzip.open('mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = cPickle.load(f, encoding = 'latin1')
    f.close()

    return (training_data, validation_data, test_data)


def load_data_wrapper():
    tr_d, va_d, te_d = load_data()

    training_inputs = [np.reshape(ti, (784, 1)) for ti in tr_d[0]]
    training_results = [vectorized_result(r) for r in tr_d[1]]
    training_data = zip(training_inputs, training_results)

    validation_inputs = [np.reshape(vi, (784, 1)) for vi in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])

    test_inputs = [np.reshape(ti, (784, 1)) for ti in te_d[0]]
    test_data = zip(test_inputs, te_d[1])

    return (training_data, validation_data, test_data)


def vectorized_result(num):
    result = np.zeros((10, 1))
    result[num] = 1.0
    return result

In [18]:
training_data, validation_data, test_data = load_data_wrapper()

In [19]:
import random
import numpy as np


class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
            
        training_data = list(training_data)
        n = len(training_data)

        for i in range(epochs):
            random.shuffle(training_data)

            mini_batches = [training_data[k:k+mini_batch_size]
                            for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)

            if test_data:
                print("Epoch {0}: {1}/{2}".format(i, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete.".format(i))

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        self.biases = [b - (eta/len(nabla_b))*nb for b, nb in zip(self.biases, nabla_b)]
        self.weights = [w - (eta/len(nabla_w))*nw for w, nw in zip(self.weights, nabla_w)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        #forward pass
        activation = x
        activations = [x]
        zs = []
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            activation = sigmoid(z)
            activations.append(activation)
            zs.append(z)

        #backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())

            return (nabla_b, nabla_w)


    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]

        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)

def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1 - sigmoid(z))

In [20]:
net = Network([784, 30, 10])

In [21]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8923/10000
Epoch 1: 9060/10000
Epoch 2: 9151/10000
Epoch 3: 9224/10000
Epoch 4: 9306/10000
Epoch 5: 9199/10000
Epoch 6: 9345/10000
Epoch 7: 9337/10000
Epoch 8: 9310/10000
Epoch 9: 9257/10000
Epoch 10: 9350/10000
Epoch 11: 9377/10000
Epoch 12: 9353/10000
Epoch 13: 9360/10000
Epoch 14: 9352/10000
Epoch 15: 9386/10000
Epoch 16: 9354/10000
Epoch 17: 9376/10000
Epoch 18: 9427/10000
Epoch 19: 9437/10000
Epoch 20: 9384/10000
Epoch 21: 9354/10000
Epoch 22: 9369/10000
Epoch 23: 9359/10000
Epoch 24: 9397/10000
Epoch 25: 9398/10000
Epoch 26: 9435/10000
Epoch 27: 9442/10000
Epoch 28: 9418/10000
Epoch 29: 9445/10000
